### <font color='e28743'>CS_5002 Final Project<font><a class='anchor' id='top'></a>
- Topic: Optimizing Project Management Using Graph Theory
- Team: Qiuying Zhuo, Zhiwei Zhou<font>

- [Step 1: Process input data](#s1) 
- [Step 2: Represent the graph using NetworkX](#s2)
- [Step 3: Find the critical path](#s3)
- [Step 4: Format and plot](#s4)
- [Step 5: Generate summarized message](#s5)



<h3 align="left"><font color='lightgreen'>Code Review</font></h3>


<p1 align="left"><font color='orange'>Step 1: Process input data </font></p1>  <a id='#s1'></a> - [back to top](#top)

To enable our further analysis using Python, we must first process the input data in a structured format. Our choice to use the CSV file format is rooted in its simplicity, ease of processing, and potential compatibility with Google Forms and other data processing tools. 

The first step involves reading and processing the data from the CSV file. We initialize dictionaries and an edge_list to store the processed data. To simplify our graph structure and analysis, we introduce virtual start and end nodes, which serve as placeholders for the beginning and end of the project timeline. These virtual nodes act as the common starting and ending points for all tasks in the project. By adding a virtual start node, we can easily connect it to all tasks that do not have any prerequisites, effectively establishing a single entry-point for the project. Similarly, the virtual end node is connected to all tasks that do not have any successors, providing a single exit point for the project. By incorporating virtual start and end nodes in our Python-based critical path analysis, we can more accurately represent the project structure and streamline the process of finding the critical path. See below figures for illustration. 

Next, we iterate through each row in the CSV file, extracting essential information such as the node label, description, duration, and preceding works. For each task, we update the id_to_name_dict with the task's node label and duration, id_duration_dict with the node label and its duration, and id_description_dict with the node label and description. 

 

We then create directed edges between tasks based on their dependencies, updating the edge_list accordingly. If a task has no preceding tasks, we add an edge from the virtual start node to the task. For tasks with preceding nodes, we add edges connecting the task to its dependencies. Finally, we add edges from tasks without successors to the virtual end node. 

 

Once the input data is processed, we have the necessary information stored in dictionaries and the edge_list, which allows us to further analyze and optimize the project using graph theory. 

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import csv

######################
# Process input data #
######################

# Read input data from CSV file and process it
filename = "event_planning.csv"

# Define virtual start and end nodes
start_node, end_node = 'VI', 'VO'

# # Initialize dictionaries and edge_list to store processed data
id_duration_dict = {start_node: 0, end_node: 0}
id_to_name_dict = {start_node: start_node, end_node: end_node}
id_description_dict = {}
edge_list = []

# Read and process CSV file
with open(filename, 'r', encoding='UTF-8') as file:
    reader = csv.reader(file)
    next(reader)  # skip header row

    for row in reader:
        # Process each row and update dictionaries and edge_list
        node_label = row[0]
        description = row[1]
        duration = int(row[2]) if row[2] else 0
        preceding = row[3].split(',') if row[3] else []
        
        # Deal with potential duplicate row inputs (as identified by id)
        if node_label in id_to_name_dict:
            continue

        # Add the node and its duration to id_to_name_dict
        id_to_name_dict[node_label] = node_label+':'+str(duration)

        # Add the node and its duration to node_duration_dict
        id_duration_dict[id_to_name_dict[node_label]] = duration

        # Add the node and its description to node_description_dict
        id_description_dict[node_label] = description

        # If node has no preceding node, add edge from virtual start node
        if not preceding:
            edge = (id_to_name_dict[start_node], id_to_name_dict[node_label])
            if edge not in edge_list:
                edge_list.append(edge)
        else:
            # If node has preceding nodes, add edges accordingly
            for p in preceding:
                s = p.strip()
                edge = (id_to_name_dict[s], id_to_name_dict[node_label])
                if edge not in edge_list:
                    edge_list.append(edge)

# If node is not being pointed to, add edge to virtual end node
for node_label in id_duration_dict.keys():
    if node_label not in [e[0] for e in edge_list] and node_label != end_node:
        edge = (node_label, id_to_name_dict[end_node])
        if edge not in edge_list:
            edge_list.append(edge)


# Print processed input data
print('\nDone with step 1: Process input data')
print('id_duration_dict:', id_duration_dict)
print('id_to_name_dict:', id_to_name_dict)
print('id_description_dict:', id_description_dict)
print('edge_list', edge_list)


 <a id='s2'></a>
 <p1 align="left"><font color='orange'>Step 2: Represent the graph using NetworkX </font></p1> - [back to top](#top)

In this step, we represent the project's task dependency graph using the NetworkX library recommended by our Professor. NetworkX can work with complex graphs and networks, making it an ideal choice for our project. The primary objective of this step is to create a directed graph object (DiGraph) and add the edges from the previously created edge_list. 

 

To begin, we instantiate a new directed graph object G and add edges to it using the add_edges_from method. This function takes the edge_list as input and adds each edge to the graph. Since we are working with a DAG, it is essential to ensure that our graph G is a valid DAG. We use the is_directed_acyclic_graph method to check if the graph is a valid DAG. This function returns a Boolean value, which is True if the graph is a valid DAG and False otherwise. 

 

Finally, we print the graph representation information to verify that our graph has been correctly constructed. The output shows that our graph G is a DiGraph with 12 nodes and 15 edges and confirms that it is a valid directed acyclic graph. By representing the graph using NetworkX, we can now leverage the library's extensive functionalities to analyze the graph and find the critical path. 

In [ ]:

######################################
# Represent the graph using networkx #
######################################

# Create a directed graph object and add edges from the edge_list
G = nx.DiGraph()
G.add_edges_from(edge_list)

# Check if G is a valid Directed Acyclic Graph (DAG)
is_valid_DAG = nx.is_directed_acyclic_graph(G)

# Print graph representation information
print('\nDone with step 2: Represent the graph using networkx')
print(f'G is now a {G}')
print(f'G is a valid directed acyclic graph: {is_valid_DAG}')


<a id = 's3'></a>
<p1 align="left"><font color='orange'>Step 3: Find the critical path / the longest path</font></p1>  - [back to top](#top)

In this step, we aim to find the critical path, which is the longest path in our DAG. As a recap, the critical path represents the sequence of tasks that take the longest time to complete, determining the minimum project duration. To accomplish this, we implement a brute-force approach using the NetworkX library. 

 

First, we define the find_longest_path function that takes a graph, a start node, and an end node as its arguments. Within this function, we initialize the longest_path as an empty list and the longest_length as negative infinity. We then iterate through all simple paths between the start and end nodes, using the all_simple_paths function. This function returns a generator of all simple paths in the graph, allowing us to efficiently process each path. The time complexity of this function is O(|V|+|E|) for each path generated, where |V| is the number of nodes and |E| is the number of edges in the graph. 

 

For each simple path, we calculate its length by summing the durations of its nodes using the id_duration_dict. If the length is greater than the current longest_length, we update the longest_length and the longest_path accordingly. Once all simple paths have been processed, the function returns the longest_path and its longest_length. 

 

Next, we call the find_longest_path function with our graph G, start_node, and end_node to determine the longest path and its length. We then update our data structures by removing the virtual nodes and edges associated with them. This removal is essential to prepare the list for formatting and plotting using matplotlib, as we want the final chart to represent the actual project tasks without the virtual points. 

 

Finally, we print the critical path information, including the longest path, its length, and its edges. By finding the critical path in our DAG, we can effectively identify the sequence of tasks that determine the minimum project duration and optimize our project management process accordingly. 

 

In [ ]:

##########################
# Find the critical path #
##########################

# Function to find the longest path (brute-force approach)
def find_longest_path(graph, start, end):
    longest_path = []
    longest_length = float('-inf')  # Initialize to negative infinity

    for path in nx.all_simple_paths(graph, start, end):
        # print(path)  # Test: 6 simple paths from VI to VO
        length = 0
        for node in path:
            length += id_duration_dict[node]
        if length > longest_length:
            longest_length = length
            longest_path = path

    return longest_path, longest_length


# Find the longest path and its length
longest_path, longest_length = find_longest_path(G, start_node, end_node)
edges_lp = list(zip(longest_path, longest_path[1:]))

# Update data structures by removing virtual nodes
id_duration_dict.pop(start_node)
id_duration_dict.pop(end_node)
edge_list = [edge for edge in edge_list if start_node not in edge and end_node not in edge]
longest_path.pop(0)
longest_path.pop(len(longest_path)-1)
edges_lp = [edge for edge in edges_lp if start_node not in edge and end_node not in edge]
G.remove_node(start_node)
G.remove_node(end_node)

# Print critical path information
print('\nDone with step 3: Find the critical path')
print('The longest path is', longest_path, 'with a length of', longest_length)
print('The edges of the longest path is', edges_lp)


<p1 align="left"><font color='orange'>Step 4: Format and plot</font></p1>  <a id='s4'></a> - [back to top](#top)

Next, we focus on visualizing the DAG representing our project tasks and the critical path. We use the Matplotlib library to create a visually appealing and informative plot. 

 

First, we create a new figure using figure function in the library. Next, we generate the node positions for our graph using the shell_layout function. We chose the shell layout after experimenting with several layout options, such as spring and spectral, and found that it provided the best visualization for our purposes. This function returns a dictionary of node positions, which we then modify by inverting the y-coordinates to achieve our desired layout. 

 

We then define the node and edge colors for our plot. Nodes belonging to the critical path are colored red, while other nodes are colored steelblue. Similarly, edges belonging to the critical path are colored red, and all other edges are colored grey. 

 

With the layout and styling defined, we proceed to draw the DAG using the draw function. We pass the necessary parameters, such as the graph G, node positions pos, node and edge colors, as well as other visual properties like label colors and node size. After drawing the graph, we save the resulting plot as a PNG file using the savefig function. We then display the plot using show function (see below) and clear the current figure with clf to prepare for any subsequent plots. 

 

Lastly, we print information about the generated plot, including the file name and storage location. By visualizing our project's DAG and the critical path, we can better understand the project structure and identify areas for potential optimization. 

In [ ]:

###################
# Format and plot #
###################

# Set layout options and draw the graph using matplotlib
plt.figure()
pos = nx.shell_layout(G)
pos = {k: (v[0], -v[1]) for k, v in pos.items()}
node_col = ['red' if node in longest_path else 'steelblue' for node in G.nodes()]
edge_colors = ['red' if edge in edges_lp else 'grey' for edge in G.edges()]

# Draw DAG
nx.draw(G, pos, with_labels=True, font_color='white', edge_color=edge_colors,
        node_color=node_col, node_size=700)

# Save graph plot to file and display it
filename = filename + '.png'
plt.savefig(filename, format="PNG")
plt.show()
plt.clf()

# Print plot information
print('\nDone with step 4: Format and plot')
print(f'Picture is stored as {filename}')



<p1 align="left"><font color='orange'>Step 5: Generate summarized message</font></p1>  <a id='s5'></a> - [back to top](#top)

In the fifth and final step of our project, we generate a summarized message to provide a clear and concise overview of the critical path and its total duration. This summary helps stakeholders understand the project timeline and the essential tasks that determine its overall duration. 

 

We create a function called generate_summary that returns the completed summary string, which is printed as the final output of our program. Below is an example of the summary output. By providing a clear and concise summary of the critical path, we offer valuable information to project stakeholders that can be used to optimize project management and resource allocation. 

In [ ]:

###########################
# Generate output message #
###########################

# Function to generate and print a summary of the critical path
def generate_summary(longest_path, longest_length, id_description_dict):
    summary = "The critical path consists of the following tasks:\n"
    
    for idx, task in enumerate(longest_path):
        task_id = task.split(':')[0]
        task_description = id_description_dict[task_id]
        task_duration = task.split(':')[1]
        summary += f"{idx + 1}. {task_id}: {task_description} ({task_duration} days)\n"
    
    summary += f"\nThe total duration of the critical path is {longest_length} days."
    
    return summary

# Function to generate and print a summary of the non-critical path
def generate_non_critical_summary(longest_path, id_description_dict):
    non_critical_summary = "The tasks NOT on the critical path are:\n"
    
    for id in id_description_dict:
        id_duration = id_to_name_dict[id]
        if id_duration not in longest_path:
            id_description = id_description_dict[id]
            id_duration = id_duration_dict[id_duration]
            non_critical_summary += f"{id}: {id_description} ({id_duration} days)\n"
    
    return non_critical_summary


# Generate the summary and print it
print(f'\nSUMMARY\n')
summary = generate_summary(longest_path, longest_length, id_description_dict)
print(summary)

# Generate the non-critical tasks summary and print it
print(f'\nREFERENCE\n')
non_critical_summary = generate_non_critical_summary(longest_path, id_description_dict)
print(non_critical_summary)